모듈 ∙ 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import FinanceDataReader as fdr
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping

데이터 불러오기 ∙ 전처리

In [2]:
def filter_df(df): #데이터프레임 필터링
    # 첫 번째 열에서 같은 값을 가진 행의 수를 계산합니다.
    row_counts = df['0'].value_counts()

    # 가장 많은 행의 수를 찾습니다.
    max_row_count = row_counts.max()

    # 가장 많은 행의 수에 해당하는 행만 분류합니다.
    filtered = df[df['0'].isin(row_counts[row_counts == max_row_count].index)]

    return filtered, max_row_count

def generator_3d_array(filtered, ar_len):

    # 데이터프레임을 NumPy 배열로 변환합니다.
    data_array = filtered.values

    # GRU 모델의 입력 데이터로 사용할 3D 배열을 초기화합니다.
    input_data = []

    # 데이터를 20 단위로 나눠서 3D 배열로 변환합니다.
    for i in range(0, len(data_array) - ar_len + 1, ar_len):
        sequence = data_array[i:i+ar_len]  # 첫번째 열을 제외한 데이터
        input_data.append(sequence)

    # input_data를 NumPy 배열로 변환합니다.
    input_data = np.array(input_data)
    
    return input_data

# 데이터 로드   
stock_df = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData_Setting.csv',low_memory=False)
pred_df = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData_Pred.csv',low_memory=False)

# #데이터 필터링
# filter_stock, stock_len = filter_df(stock_df)
# filter_pred,  pred_len= filter_df(pred_df)

#예측에 필요한 데이터 
pred_ticker = pd.DataFrame(pred_df['0'])#종목코드 저장
stock_label = pd.DataFrame(stock_df['18']) #5% 이상 상승여부  

#불필요한 데이터 삭제
filter_stock = stock_df.drop({'0','1','7','18'},axis=1) #날자, 상승율, 5%이상 상승여부 삭제 

filter_pred = pred_df.drop({'0','1','7'},axis=1)#날자, 상승율 삭제

# stock_3d = generator_3d_array(filter_stock, stock_len)
# pred_3d = generator_3d_array(filter_pred, pred_len)


데이터 전처리

In [3]:
scaler = MinMaxScaler()
stock_label = pd.DataFrame(scaler.fit_transform(stock_label.values.reshape(-1, 1)))

# label_3d = generator_3d_array(stock_label,stock_len)

데이터 분할

In [4]:
# 훈련 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(filter_stock, stock_label, test_size=0.2, random_state=42)

GRU 모델 정의

In [5]:
# GRU 모델 정의
model = tf.keras.Sequential()
model.add(tf.keras.layers.GRU(128, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(tf.keras.layers.GRU(512))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.summary()

2023-10-22 23:23:21.780913: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-22 23:23:21.780940: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-10-22 23:23:21.780946: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-10-22 23:23:21.780993: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-22 23:23:21.781014: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 28, 128)           50304     
                                                                 
 gru_1 (GRU)                 (None, 512)               986112    
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 1036929 (3.96 MB)
Trainable params: 1036929 (3.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


모델 훈련

In [6]:
# 모델 훈련
model.fit(X_train, y_train, epochs=50)
#, validation_data=(X_test,y_test)
# callbacks=[early_stopping]
# 모델 저장
model.save("GRU_Model.h5")

Epoch 1/50


2023-10-22 23:23:24.154585: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-22 23:23:24.482203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-22 23:23:25.009760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-22 23:23:27.297011: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-22 23:23:28.372177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 1466/22324 [>.............................] - ETA: 7:00 - loss: 0.1302 - accuracy: 0.9629

KeyboardInterrupt: 

모델 평가

In [ ]:
# 모델 로드
loaded_model = tf.keras.models.load_model("GRU_Model.h5")

# 모델을 사용하여 주가 상승 여부 예측
test = loaded_model.predict(X_test)

In [ ]:
# 예측 결과를 이진값(0 또는 1)으로 변환
binary_predictions = (test > 0.5).astype(int)

# 예측 결과 출력
correct_predictions = np.equal(binary_predictions, y_test)  # 정확하게 예측한 경우 True, 그렇지 않으면 False
accuracy = np.mean(correct_predictions)  # 정확도 계산

print(f"정확도: {accuracy * 100:.2f}%")

예측

In [ ]:
def make_six_digit_list(input_list):
    six_digit_list = []

    for item in input_list:
        # 현재 항목의 길이를 확인
        item_str = str(item)
        item_len = len(item_str)

        if item_len < 6:
            # 6자리가 안되는 항목은 앞에 0을 채워서 6자리로 만듭니다.
            zero_padding = '0' * (6 - item_len)
            six_digit_item = zero_padding + item_str
            six_digit_list.append(six_digit_item)
        else:
            # 이미 6자리인 경우 그대로 유지
            six_digit_list.append(item_str)

    return six_digit_list

In [ ]:
# 모델을 사용하여 주가 상승 여부 예측
T_pred = loaded_model.predict(pred_3d)
# # 5% 이상 오를 종목 식별
# threshold = 0.5
# rising_stocks = [ticker for i, ticker in enumerate(pred_ticker) if T_pred[i] > threshold]
# rising_stocks = list(map(str, rising_stocks))
# result = list(set(rising_stocks))
# result = make_six_digit_list(result)
# print("내일 5% 이상 상승할 종목:", result)
# print(len(result))

예측한 결과 검증

In [ ]:
def check_price_increase(stock_codes):
    result = {}
    
    for code in stock_codes:
        df = fdr.DataReader(code,'2023-10-12','2023-10-13')
        
        if len(df) < 2:
            result[code] = False
        else:
            today_close = df['Close'].iloc[-1]
            yesterday_close = df['Close'].iloc[-2]
            
            if today_close >= yesterday_close * 1.05 :
                result[code] = True
            else:
                result[code] = False
    return result

result_TF = check_price_increase(result)
result_TF = list(result_TF.values())

# True와 False의 개수를 세어 비율을 계산
true_count = sum(result_TF)
false_count = len(result_TF) - true_count

# 비율을 파이그래프로 표현
labels = ['True', 'False']
sizes = [true_count, false_count]
colors = ['green', 'red']

# 파이차트 그리기
plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)

# 그래프 표시
plt.show() 